In [1]:
import pandas as pd
import datetime

In [2]:
#read in datasets
clean_crime = pd.read_csv('../clean_datasets/clean_crime.csv')
area_reference = pd.read_csv('../scoring_datasets/area_reference.csv')
clean_ridership = pd.read_csv('../clean_datasets/clean_ridership.csv')
clean_lighting = pd.read_csv('../clean_datasets/clean_lighting.csv')
clean_divvy_trips = pd.read_csv('../clean_datasets/clean_divvy_trips.csv')
clean_vacant_buildings = pd.read_csv('../clean_datasets/clean_vacant_buildings.csv')

In [3]:
def fill_gaps(df):
    all_dates = pd.date_range(start='2001-01-01', end='12/31/2022', freq='D')
    all_hours = range(1, 24)
    all_areas = range(1, 78)
    df['date'] = pd.to_datetime(df['date'])
    date_hour_combinations = pd.DataFrame(([datetime.datetime.strptime(str(date)[0:10], "%Y-%m-%d"), hour, area] for date in all_dates for hour in all_hours for area in all_areas), columns=['date', 'hour', 'area'])
    merged_df = date_hour_combinations.merge(df, on=['date', 'hour', 'area'], how='outer').fillna(0)

    return merged_df

In [4]:
def aggregate_data():

    #clean the crime data so the aggregation process is smoother
    clean_crime['date'] = [datetime.datetime.strptime(clean_crime.date.iloc[i], '%m/%d/%Y %I:%M:%S %p') for i in range(len(clean_crime))]
    clean_crime['hour'] = clean_crime.date.apply(lambda x : (x.hour + 1))
    clean_crime['date'] = clean_crime.date.apply(lambda x : x.date())

    violent_crimes = [ 'BATTERY', 'ASSAULT', 'CRIM SEXUAL ASSAULT', 'SEX OFFENSE', 'WEAPONS VIOLATION',
    'HOMICIDE', 'KIDNAPPING', 'ROBBERY', 'INTIMIDATION', 'ARSON', 'CRIMINAL SEXUAL ASSAULT', 'HUMAN TRAFFICKING']

    clean_crime['type'] = clean_crime['type'].apply(lambda x : 2 if x in violent_crimes else 1)

    print('clean_crime successfully read in')

    #group the crime dataset utilizing a count aggregation
    grouped_crime = clean_crime.groupby(['date', 'hour', 'type', 'area']).size().reset_index(name='count')

    #pivot the violent vs non-violent counts to display across our grouped columns
    grouped_crime = grouped_crime.pivot_table(index=['date', 'hour', 'area'], columns='type', values='count', fill_value=0).reset_index()
    grouped_crime.rename(columns={1 : "non-violent", 2 : 'violent'}, inplace=True)

    print('crime dataset successfully grouped')

    #call fill_gaps function
    grouped_crime = fill_gaps(grouped_crime)

    print('fill gaps function successfully ran in')
    
    #merge the area_reference dataset on top of the grouped_crime data
    grouped_crime = grouped_crime.merge(area_reference, left_on='area', right_on='id', how='left')
    grouped_crime.drop(columns=['id'], inplace=True)

    print('area reference data successfully merged')
    
    #match datatypes of columns and then merge the two datasets together
    grouped_crime['date'] = pd.to_datetime(grouped_crime['date'])
    clean_ridership['date'] = pd.to_datetime(clean_ridership['date'])
    grouped_crime = grouped_crime.merge(clean_ridership, on=['date', 'area'], how='left')
    
    print('clean ridership data successfully merged')

    #match datatypes of columns and then group the dataset to make merging together easier
    clean_divvy_trips['hour'] = pd.to_datetime(clean_divvy_trips['date']).dt.hour
    clean_divvy_trips['date'] = pd.to_datetime(pd.to_datetime(clean_divvy_trips['date']).dt.date)
    grouped_divvy = clean_divvy_trips.groupby(['date', 'hour', 'area'])['station_id'].agg('count').reset_index()  

    #merge the grouped divvy data onto the base crime data  
    grouped_crime = grouped_crime.merge(grouped_divvy, on=['date', 'hour', 'area'], how='left')
    grouped_crime.rename(columns={'station_id' : 'bike_rides', 'rides' : 'train_rides'}, inplace=True)

    print('clean divvy trips data successfully merged')

    grouped_lighting = clean_lighting.groupby(['date', 'area'])['lat'].agg('count').reset_index()
    grouped_lighting.date = pd.to_datetime(grouped_lighting.date)
    grouped_crime = grouped_crime.merge(grouped_lighting, on=['date', 'area'], how='left')
    grouped_crime.rename(columns={'lat' : 'lighting'}, inplace=True)

    print('clean lighting data successfully merged')

    clean_vacant_buildings.date = pd.to_datetime(clean_vacant_buildings.date)
    grouped_vacancies = clean_vacant_buildings[clean_vacant_buildings.date <= pd.to_datetime('2022-12-31')].groupby(['date', 'area'])['long'].agg('count').reset_index()
    grouped_crime = grouped_crime.merge(grouped_vacancies, on=['date', 'area'], how='left')
    grouped_crime.rename(columns={'long' : 'vacant_buildings'}, inplace=True)

    print('clean vacancies data successfully merged')

    return grouped_crime


In [5]:
final_aggregation = aggregate_data()
final_aggregation

clean_crime successfully read in
crime dataset successfully grouped
fill gaps function successfully ran in
area reference data successfully merged
clean ridership data successfully merged
clean divvy trips data successfully merged
clean lighting data successfully merged
clean vacancies data successfully merged


,date,hour,area,non-violent,violent,cta_stations,police_stations,bus_stations,unemployment,per_capita_income,no_hs_dip,gov_depend,crowded_housing,below_pov,bike_stations,train_rides,bike_rides,lighting,vacant_buildings
0,2001-01-01,1,1,4.0,0.0,4.0,1.0,7.0,0.075,23714.0,0.181,0.288,0.079,0.227,17.0,NaN,NaN,NaN,NaN
1,2001-01-01,1,2,3.0,0.0,0.0,0.0,2.0,0.079,21375.0,0.196,0.383,0.070,0.151,30.0,NaN,NaN,NaN,NaN
2,2001-01-01,1,3,5.0,2.0,3.0,0.0,3.0,0.077,32355.0,0.136,0.222,0.046,0.227,15.0,NaN,NaN,NaN,NaN
3,2001-01-01,1,4,2.0,0.0,4.0,1.0,5.0,0.068,35503.0,0.125,0.256,0.031,0.095,19.0,NaN,NaN,NaN,NaN
4,2001-01-01,1,5,1.0,0.0,2.0,0.0,5.0,0.045,51615.0,0.054,0.255,0.002,0.071,15.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14611310,2023-10-10,1,33,1.0,0.0,2.0,1.0,7.0,0.057,60593.0,0.071,0.210,0.014,0.111,15.0,NaN,NaN,NaN,NaN
14611311,2023-10-10,1,42,1.0,0.0,2.0,1.0,9.0,0.173,18928.0,0.179,0.376,0.018,0.283,22.0,NaN,NaN,NaN,NaN
14611312,2023-10-10,1,43,1.0,0.0,0.0,0.0,10.0,0.177,18366.0,0.149,0.376,0.029,0.315,23.0,NaN,NaN,NaN,NaN
14611313,2023-10-10,1,44,1.0,0.0,2.0,0.0,10.0,0.190,20320.0,0.137,0.400,0.022,0.253,33.0,NaN,NaN,NaN,NaN


In [6]:
final_aggregation.to_csv('../scoring_datasets/final_aggregation.csv', index=False)